In [1]:
!pip install happybase

     |████████████████████████████████| 40 kB 2.0 MB/s eta 0:00:011
     |████████████████████████████████| 361 kB 1.5 MB/s eta 0:00:01
     |████████████████████████████████| 49 kB 3.1 MB/s eta 0:00:01
  Created wheel for happybase: filename=happybase-1.2.0-py2.py3-none-any.whl size=26606 sha256=adc34fc695ab839ecef557c6db86b020f96ac28bfef8543ae41329a1b4d5c08b
  Stored in directory: /home/jovyan/.cache/pip/wheels/9e/00/b2/3a44a68da0a23b4f274c24f3bda10e84283d63fb8b8dfb3709
  Created wheel for thriftpy2: filename=thriftpy2-0.4.14-cp39-cp39-linux_x86_64.whl size=507857 sha256=f8b52683200903b024541ce4dd180896e03dc97a340d468a1fa1b1d38c791b6e
  Stored in directory: /home/jovyan/.cache/pip/wheels/a1/97/73/c37a0f04483972cb2ca32764534da03dcfec3b2f0f8855a87d
Successfully built happybase thriftpy2


In [2]:
!pip install thrift

     |████████████████████████████████| 59 kB 1.0 MB/s eta 0:00:011
  Created wheel for thrift: filename=thrift-0.13.0-cp39-cp39-linux_x86_64.whl size=182896 sha256=5ac3e85b98ab627a6830175be4989f05c7775fa6ee4d9e2b05a783660fe5b03c
  Stored in directory: /home/jovyan/.cache/pip/wheels/a3/18/80/a9b97f8e0dddb23be3e79f2209dfa3cd6292e1e42bd210a068
Successfully built thrift


In [3]:
import happybase

connection = happybase.Connection('hbase-master')
print(connection.tables())

[b'my_table', b'person']


In [11]:
#建立一個table
connection.create_table(
    'my_table',
    {
        'cf1': dict(max_versions=10), #版本保留10個
        'cf2': dict(max_versions=1, block_cache_enabled=False), #不進行緩存
        'cf3': dict()  #使用預設值
    }
)

In [4]:
print(connection.tables())

[b'my_table', b'person']


In [5]:
table = connection.table('my_table')

In [9]:
#若RowKey已經存在，就會變成更新資料

sensor1 = {'cf1:temperature': '36.5', 'cf1:humidity': '56', 'cf1:vibration': '265'}
sensor2 = {'cf1:temperature': '36.7', 'cf1:humidity': '56', 'cf1:vibration': '287'}
sensor3 = {'cf1:temperature': '36.8', 'cf1:humidity': '57', 'cf1:vibration': '289'}
sensor4 = {'cf2:voltage': '220', 'cf2:current': '6'}

#這樣會put四次
table.put(row='machine1', data=sensor1)
table.put(row='machine2', data=sensor2)
table.put(row='machine3', data=sensor3)
table.put(row='machine4', data=sensor4)

In [14]:
#查看
for key, value in table.scan():
    print(key, value)

b'machine1' {b'cf1:humidity': b'56', b'cf1:temperature': b'36.5', b'cf1:vibration': b'265'}
b'machine2' {b'cf1:humidity': b'56', b'cf1:temperature': b'36.7', b'cf1:vibration': b'287'}
b'machine3' {b'cf1:humidity': b'57', b'cf1:temperature': b'36.8', b'cf1:vibration': b'289'}
b'machine4' {b'cf2:current': b'6', b'cf2:voltage': b'220'}
b'machine5' {b'cf1:temperature': b'37', b'cf2:humidity': b'55', b'cf2:voltage': b'220'}
b'machine6' {b'cf1:temperature': b'37', b'cf2:current': b'8', b'cf2:humidity': b'55'}


In [13]:
#批次
bat = table.batch()
bat.put('machine5', {'cf1:temperature': '37', 'cf2:humidity': '55', 'cf2:voltage': '220'})
bat.put('machine6', {'cf1:temperature': '37', 'cf2:humidity': '55', 'cf2:current': '8'})
bat.send()

In [15]:
#查看
for key, value in table.scan():
    print(key, value)

b'machine1' {b'cf1:humidity': b'56', b'cf1:temperature': b'36.5', b'cf1:vibration': b'265'}
b'machine2' {b'cf1:humidity': b'56', b'cf1:temperature': b'36.7', b'cf1:vibration': b'287'}
b'machine3' {b'cf1:humidity': b'57', b'cf1:temperature': b'36.8', b'cf1:vibration': b'289'}
b'machine4' {b'cf2:current': b'6', b'cf2:voltage': b'220'}
b'machine5' {b'cf1:temperature': b'37', b'cf2:humidity': b'55', b'cf2:voltage': b'220'}
b'machine6' {b'cf1:temperature': b'37', b'cf2:current': b'8', b'cf2:humidity': b'55'}


In [16]:
#刪除
with table.batch() as bat:
    bat.put('machine7', {'cf1:temperature': '39', 'cf2:humidity': '45', 'cf2:voltage': '110'})
    bat.delete('machine5')

In [17]:
#查看
for key, value in table.scan():
    print(key, value)

b'machine1' {b'cf1:humidity': b'56', b'cf1:temperature': b'36.5', b'cf1:vibration': b'265'}
b'machine2' {b'cf1:humidity': b'56', b'cf1:temperature': b'36.7', b'cf1:vibration': b'287'}
b'machine3' {b'cf1:humidity': b'57', b'cf1:temperature': b'36.8', b'cf1:vibration': b'289'}
b'machine4' {b'cf2:current': b'6', b'cf2:voltage': b'220'}
b'machine6' {b'cf1:temperature': b'37', b'cf2:current': b'8', b'cf2:humidity': b'55'}
b'machine7' {b'cf1:temperature': b'39', b'cf2:humidity': b'45', b'cf2:voltage': b'110'}


In [18]:
#通過row_start引數來設定開始掃描的RowKey
for key, value in table.scan(row_start='machine3'):
    print(key, value)

b'machine3' {b'cf1:humidity': b'57', b'cf1:temperature': b'36.8', b'cf1:vibration': b'289'}
b'machine4' {b'cf2:current': b'6', b'cf2:voltage': b'220'}
b'machine6' {b'cf1:temperature': b'37', b'cf2:current': b'8', b'cf2:humidity': b'55'}
b'machine7' {b'cf1:temperature': b'39', b'cf2:humidity': b'45', b'cf2:voltage': b'110'}


In [19]:
#通過row_stop引數來設定結束掃描的RowKey
for key, value in table.scan(row_stop='machine3'):
    print(key, value)

b'machine1' {b'cf1:humidity': b'56', b'cf1:temperature': b'36.5', b'cf1:vibration': b'265'}
b'machine2' {b'cf1:humidity': b'56', b'cf1:temperature': b'36.7', b'cf1:vibration': b'287'}


In [21]:
#通過row_start和row_stop引數來設定開始和結束掃描的RowKey
for key, value in table.scan(row_start='machine1', row_stop='machine5'):
    print(key, value)

b'machine1' {b'cf1:humidity': b'56', b'cf1:temperature': b'36.5', b'cf1:vibration': b'265'}
b'machine2' {b'cf1:humidity': b'56', b'cf1:temperature': b'36.7', b'cf1:vibration': b'287'}
b'machine3' {b'cf1:humidity': b'57', b'cf1:temperature': b'36.8', b'cf1:vibration': b'289'}
b'machine4' {b'cf2:current': b'6', b'cf2:voltage': b'220'}


In [32]:
#查看資料
row = table.row('machine1')
res=row
res

{b'cf1:humidity': b'56', b'cf1:temperature': b'36.5', b'cf1:vibration': b'265'}

In [43]:
#抓出資料
byte_humidity=res[b'cf1:humidity']

#將byte轉為string
encoding = 'utf-8'
humidity=str(byte_humidity, encoding)
print(humidity)

56


In [46]:
#查看多筆資料
rows = dict(table.rows(['machine1', 'machine2']))
print(rows)

{b'machine1': {b'cf1:humidity': b'56', b'cf1:temperature': b'36.5', b'cf1:vibration': b'265'}, b'machine2': {b'cf1:humidity': b'56', b'cf1:temperature': b'36.7', b'cf1:vibration': b'287'}}


In [59]:
#讀取
for key, value in rows.items():
    print(str(value[b'cf1:humidity'], 'utf-8'))

56
56


In [61]:
#通過指定條件查找資料
row = table.row('machine6', columns=['cf1'])
print(row)

{b'cf1:temperature': b'37'}


In [65]:
#通過指定條件查找資料
row = table.row('machine6', columns=[b'cf1:temperature', b'cf2:current'])
print(row)

{b'cf1:temperature': b'37', b'cf2:current': b'8'}


In [83]:
import time

#date to Timestamp
def Timestamp_conversion(date):
    timeArray = time.strptime(date, "%Y-%m-%d %H:%M:%S")
    timestamp = int(time.mktime(timeArray))
    return timestamp
    
#Timestamp to date
def date_conversion(timestamp):
    time_local = time.localtime(timestamp)
    date = time.strftime("%Y-%m-%d %H:%M:%S",time_local)
    return date

In [69]:
#時間戳記
row = table.row(row='machine1', include_timestamp=True)
print(row)

#透過取得時間轉換
#https://www.cadch.com/article/timestamp/index.php   

{b'cf1:humidity': (b'56', 1625073688468), b'cf1:temperature': (b'36.5', 1625073688468), b'cf1:vibration': (b'265', 1625073688468)}


In [70]:
#檢視某一個cell所有版本
cells = table.cells(b'machine1', column='cf1:temperature')
print(cells)

[b'36.5', b'36.5', b'36.5']


In [71]:
#更新資料
sensor1 = {'cf1:temperature': '39.5'}
table.put(row='machine1', data=sensor1)

In [78]:
#再次檢視cell版本
cells = table.cells(b'machine1', column=b'cf1:temperature')
print(cells)

[]


In [74]:
#通過引數來檢索前n個版本
cells = table.cells(b'machine1', column=b'cf1:temperature', versions=3)
print(cells)

[b'39.5', b'36.5', b'36.5']


In [75]:
#刪除整行資料
table.delete('machine1')

In [76]:
#刪除一個cf資料
table.delete('machine2', columns=[b'cf1'])

In [80]:
#刪除一個cf多個資料
table.delete('machine7', columns=[b'cf1:temperature',b'cf2:current'])

In [81]:
#查看
for key, value in table.scan():
    print(key, value)

b'machine3' {b'cf1:humidity': b'57', b'cf1:temperature': b'36.8', b'cf1:vibration': b'289'}
b'machine4' {b'cf2:current': b'6', b'cf2:voltage': b'220'}
b'machine6' {b'cf1:temperature': b'37', b'cf2:current': b'8', b'cf2:humidity': b'55'}
b'machine7' {b'cf2:humidity': b'45', b'cf2:voltage': b'110'}


In [88]:
connection.close()